Created by:
- Marcus Vinicius da Silva Fernandes
- Yamini Sharma

2023-06-05

In [1]:
import numpy as np
import os
import csv
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

## Loading the .npy files.

In [2]:
# Set up of the videos path
videos_path = '/Users/marcus/ASL/first_run/videos/'

# Set up of the extracted landmarks save path
landmarks_path = '/Users/marcus/ASL/first_run/landmarks/'

Creation of the dictionary to associate the videos and the words.

In [3]:
# Opening the file videos_id.csv to load the association of video ids to words
id_dict = {}  # initializing the dictionary that will receive the data
with open(videos_path + "videos_id.csv", "r") as csv_file:
    csv_reader = csv.reader(csv_file)  # reading the data
    next(csv_reader)  # to skip the header
    for row in csv_reader:
        id_dict[row[1]] = row[0]  # storing the content into a dictionary

In [4]:
id_dict

{'69241': 'book',
 '65225': 'book',
 '68011': 'book',
 '68208': 'book',
 '68012': 'book',
 '70212': 'book',
 '70266': 'book',
 '07085': 'book',
 '07086': 'book',
 '07087': 'book',
 '07069': 'book',
 '07088': 'book',
 '07089': 'book',
 '07090': 'book',
 '07091': 'book',
 '07092': 'book',
 '07093': 'book',
 '07068': 'book',
 '07094': 'book',
 '07095': 'book',
 '07096': 'book',
 '07097': 'book',
 '07070': 'book',
 '07098': 'book',
 '07099': 'book',
 '07071': 'book',
 '07072': 'book',
 '07073': 'book',
 '67424': 'book',
 '07074': 'book',
 '07075': 'book',
 '07076': 'book',
 '07077': 'book',
 '07078': 'book',
 '07079': 'book',
 '07080': 'book',
 '07081': 'book',
 '07082': 'book',
 '07083': 'book',
 '07084': 'book',
 '69302': 'drink',
 '65539': 'drink',
 '70173': 'drink',
 '68538': 'drink',
 '68042': 'drink',
 '68660': 'drink',
 '68041': 'drink',
 '17725': 'drink',
 '17726': 'drink',
 '17727': 'drink',
 '17728': 'drink',
 '17710': 'drink',
 '17729': 'drink',
 '17730': 'drink',
 '17731': 'dri

Creating the X_train array and the y_train array

In [5]:
video_labels = []
for item in os.listdir(landmarks_path):
    if item.endswith('.npy'):  # working with npy files only
        data = np.load(os.path.join(landmarks_path, item))
        if data.shape[0] == 30:
            video_labels.append(item.split('.npy')[0])
y_dict = {label:num for num, label in enumerate(video_labels)}

In [6]:
videos, labels = [], []

for item in os.listdir(landmarks_path):
    if item.endswith('.npy'):  # working with npy files only
        # print(item)
        data = np.load(os.path.join(landmarks_path, item))
        # print(data.shape)
        # X = np.vstack((X, data))
        if data.shape[0] == 30:
            videos.append(data)
            print(np.array(videos).shape)
            labels.append(y_dict[item.split('.npy')[0]])

X = np.array(videos)

(1, 30, 1662)
(2, 30, 1662)
(3, 30, 1662)
(4, 30, 1662)
(5, 30, 1662)
(6, 30, 1662)
(7, 30, 1662)
(8, 30, 1662)
(9, 30, 1662)
(10, 30, 1662)
(11, 30, 1662)
(12, 30, 1662)
(13, 30, 1662)
(14, 30, 1662)
(15, 30, 1662)
(16, 30, 1662)
(17, 30, 1662)
(18, 30, 1662)
(19, 30, 1662)
(20, 30, 1662)
(21, 30, 1662)
(22, 30, 1662)
(23, 30, 1662)
(24, 30, 1662)
(25, 30, 1662)
(26, 30, 1662)
(27, 30, 1662)
(28, 30, 1662)
(29, 30, 1662)
(30, 30, 1662)
(31, 30, 1662)
(32, 30, 1662)
(33, 30, 1662)
(34, 30, 1662)
(35, 30, 1662)
(36, 30, 1662)
(37, 30, 1662)
(38, 30, 1662)
(39, 30, 1662)
(40, 30, 1662)
(41, 30, 1662)
(42, 30, 1662)
(43, 30, 1662)
(44, 30, 1662)
(45, 30, 1662)
(46, 30, 1662)
(47, 30, 1662)
(48, 30, 1662)
(49, 30, 1662)
(50, 30, 1662)
(51, 30, 1662)
(52, 30, 1662)
(53, 30, 1662)
(54, 30, 1662)
(55, 30, 1662)
(56, 30, 1662)
(57, 30, 1662)
(58, 30, 1662)
(59, 30, 1662)
(60, 30, 1662)
(61, 30, 1662)
(62, 30, 1662)
(63, 30, 1662)
(64, 30, 1662)
(65, 30, 1662)
(66, 30, 1662)
(67, 30, 1662)
(68,

In [7]:
from tensorflow.keras.utils import to_categorical

In [8]:
Y = to_categorical(labels).astype(int)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1)

LSTM model

In [10]:
from tensorflow.keras.callbacks import TensorBoard

model = Sequential()
model.add(LSTM(64, activation='sigmoid', input_shape=(30,1662)))
model.add(Dense(y_train.shape[1], activation='softmax'))

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

model.fit(X_train, y_train, epochs=3000, callbacks=[tb_callback])

Epoch 1/3000


2023-07-03 17:59:43.080983: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


3/3 [==============================] - 4s 1s/step - loss: 4.6920 - categorical_accuracy: 0.0000e+00
Epoch 2/3000
3/3 [==============================] - 3s 898ms/step - loss: 4.6082 - categorical_accuracy: 0.0111
Epoch 3/3000
3/3 [==============================] - 3s 897ms/step - loss: 4.5799 - categorical_accuracy: 0.0111
Epoch 4/3000
3/3 [==============================] - 3s 881ms/step - loss: 4.5669 - categorical_accuracy: 0.0222
Epoch 5/3000
3/3 [==============================] - 3s 878ms/step - loss: 4.5588 - categorical_accuracy: 0.0111
Epoch 6/3000
3/3 [==============================] - 3s 907ms/step - loss: 4.5493 - categorical_accuracy: 0.0000e+00
Epoch 7/3000
3/3 [==============================] - 3s 931ms/step - loss: 4.5421 - categorical_accuracy: 0.0111
Epoch 8/3000
3/3 [==============================] - 3s 1s/step - loss: 4.5327 - categorical_accuracy: 0.0111
Epoch 9/3000
3/3 [==============================] - 3s 972ms/step - loss: 4.5236 - categorical_accuracy: 0.0111
Epo

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                442112    
                                                                 
 dense (Dense)               (None, 100)               6500      
                                                                 
Total params: 448,612
Trainable params: 448,612
Non-trainable params: 0
_________________________________________________________________


In [12]:
# model.save('first_run.h5')

Prediction

In [13]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [14]:
yhat = model.predict(X_test)

1/1 [==============================] - 0s 367ms/step


In [15]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [16]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[9., 1.],
        [0., 0.]],

       [[9., 0.],
        [1., 0.]],

       [[9., 1.],
        [0., 0.]],

       [[9., 0.],
        [1., 0.]],

       [[9., 0.],
        [1., 0.]],

       [[9., 1.],
        [0., 0.]],

       [[9., 0.],
        [1., 0.]],

       [[9., 1.],
        [0., 0.]],

       [[9., 0.],
        [1., 0.]],

       [[9., 1.],
        [0., 0.]],

       [[9., 0.],
        [1., 0.]],

       [[9., 0.],
        [1., 0.]],

       [[9., 1.],
        [0., 0.]],

       [[9., 0.],
        [1., 0.]],

       [[9., 1.],
        [0., 0.]],

       [[9., 0.],
        [1., 0.]],

       [[9., 1.],
        [0., 0.]],

       [[9., 0.],
        [1., 0.]],

       [[9., 1.],
        [0., 0.]],

       [[9., 1.],
        [0., 0.]]])

In [17]:
accuracy_score(ytrue, yhat)

0.0

model.predict(x_test)
result = model.predict(X_test)
np.argmax(result)
id_dict[video_labels[np.argmax(result)]]

print("0 = " + id_dict[video_labels[0]])
print("1 = " + id_dict[video_labels[1]])
print("2 = " + id_dict[video_labels[2]])
print("3 = " + id_dict[video_labels[3]])
print("4 = " + id_dict[video_labels[4]])
print("5 = " + id_dict[video_labels[5]])
print('Test video expected result = ' + id_dict[test_video_name])
print('Model result = ' + id_dict[video_labels[np.argmax(result)]])